# 상장기업정보 수집하기

In [3]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [4]:
dict(method=searchCorpList, pageIndex=1, currentPageSize=15, comAbbrv=, beginIndex=, orderMode=3, orderStat=D, isurCd=, repIsuSrtCd=, searchCodeType=, marketType=, searchType=13, industry=, fiscalYearEnd=all, comAbbrvTmp=, location=all)

SyntaxError: invalid syntax (4079214429.py, line 1)

In [ ]:
# post정보를 찾을라면 개발자 도구에서 주소(header,payload) 가져오고 자료 수집할때 post,data로 바꾸어서 불러오기

In [5]:
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList", pageIndex=1, currentPageSize=100,  orderMode=3, orderStat="D", searchType=13,fiscalYearEnd="all", location="all")
r = requests.post(url, data=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content,'lxml')
soup

https://kind.krx.co.kr/corpgeneral/corpList.do
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="명인제약(주)"><img alt="유가증권" class="vmiddle legend" src="/images/common/icn_t_yu.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('31745'); return false;" title="명인제약"> 명인제약</a> </td>
<td class="textOverflow" title="의약품 제조업">의약품 제조업</td>
<td class="textOverflow" title="신경정신계열, 구강계열 완제의약품">신경정신계열, 구강계열 완제의약품</td>
<td class="txc">2025-10-01</td>
<td class="txc">12월</td>
<td class="txc" title="이행명">이행명</td>
<td class="txc">
<a class="btn ico homepage" href="http://www.myunginph.co.kr" target="_blank" title="http://www.myunginph.co.kr"><sp

In [25]:
total_page = int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1
total_page

28

In [30]:
# 증권종류
soup.select_one(".vmiddle.legend")['alt']

'유가증권'

In [33]:
# 회사명
soup.select_one("#companysum")["title"]

'명인제약'

In [53]:
# 종목코드
code = soup.select_one("#companysum")["onclick"].split("'")[1]
code = f'{code:0<6s}'
code

'317450'

In [86]:
#업종
soup.select_one(".textOverflow").text

'의약품 제조업'

In [120]:
#주요제품
soup.select_one("tr.first>td:nth-child(2)").text

'의약품 제조업'

In [121]:
#업종
soup.select_one("tr.first>td:nth-child(3)").text

'신경정신계열, 구강계열 완제의약품'

In [122]:
# 상장일
soup.select_one("tr.first>td:nth-child(4)").text

'2025-10-01'

In [123]:
# 결산일
soup.select_one("tr.first>td:nth-child(5)")

<td class="txc">12월</td>

In [124]:
#대표자명
soup.select_one("tr.first>td:nth-child(6)").text

'이행명'

In [130]:
#홈페이지
soup.select_one("tr.first>td:nth-child(7)>a")['href']

'http://www.myunginph.co.kr'

In [ ]:
#main-contents > section.scrarea.type-00 > table > tbody > tr.first > td:nth-child(7) > a

In [ ]:
# 지역
soup.select_one("tr.first>td:nth-child(8)").text

In [260]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [261]:
def info_extraction(trs):
    page_result=[]
    for tr in trs:
        # 증권종류
        types = tr.select_one(".vmiddle.legend")['alt']
        # 회사명
        name = tr.select_one("#companysum")["title"]
        # 종목코드
        code = tr.select_one("#companysum")["onclick"].split("'")[1]
        code = f'{code:0<6s}'
        #업종
        type_bi = tr.select_one(".textOverflow").text
        #주요제품
        main = tr.select_one("td:nth-child(2)").text
        # 상장일
        date = tr.select_one("td:nth-child(4)").text
        # 결산일
        sum_date= tr.select_one("td:nth-child(5)").text
        #대표자명
        ceo = tr.select_one("td:nth-child(6)").text
        #홈페이지
        link = tr.select_one("td:nth-child(7)>a")['href'] if tr.select_one("tr.first>td:nth-child(7)>a") != None else '없음'
        # 지역
        do = tr.select_one("td:nth-child(8)").text
        values = (types,name,code,type_bi,main,date,sum_date,ceo,link,do)
        page_result.append(values)
    return page_result
page_result

[('유가증권',
  '명인제약',
  '317450',
  '의약품 제조업',
  '의약품 제조업',
  '2025-10-01',
  '12월',
  '이행명',
  'http://www.myunginph.co.kr',
  '경기도'),
 ('코스닥',
  'KB제33호스팩',
  '0072Z0',
  '금융 지원 서비스업',
  '금융 지원 서비스업',
  '2025-09-30',
  '12월',
  '박성원',
  '없음',
  '서울특별시'),
 ('코스닥',
  '삼성스팩11호',
  '0071M0',
  '금융 지원 서비스업',
  '금융 지원 서비스업',
  '2025-09-30',
  '12월',
  '문소연',
  '없음',
  '서울특별시'),
 ('코스닥',
  '에스투더블유',
  '488280',
  '소프트웨어 개발 및 공급업',
  '소프트웨어 개발 및 공급업',
  '2025-09-19',
  '12월',
  '서상덕',
  '없음',
  '경기도'),
 ('코스닥',
  '오아',
  '342870',
  '가전제품 및 정보통신장비 소매업',
  '가전제품 및 정보통신장비 소매업',
  '2025-09-09',
  '12월',
  '김상무',
  '없음',
  '서울특별시'),
 ('코넥스',
  '본시스템즈',
  '496320',
  '특수 목적용 기계 제조업',
  '특수 목적용 기계 제조업',
  '2025-09-08',
  '12월',
  '김창현',
  '없음',
  '경상남도'),
 ('코스닥',
  '그래피',
  '318060',
  '합성고무 및 플라스틱 물질 제조업',
  '합성고무 및 플라스틱 물질 제조업',
  '2025-08-25',
  '12월',
  '심운섭',
  '없음',
  '서울특별시'),
 ('코스닥',
  '애드포러스',
  '397810',
  '광고업',
  '광고업',
  '2025-08-22',
  '12월',
  '이은현',
  '없음',
  '서울특별시'),
 ('코스닥',
  '

In [262]:
keys = ['증권종류','회사명','종목코드','업종','주요제품','상장일','결산일','대표자명','홈페이지','지역']
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100,  orderMode=3, orderStat="D", searchType=13,fiscalYearEnd="all", location="all")
r = requests.post(url, data=payload)
soup = bs(r.content,'lxml')
page_result = info_extraction(soup.select('tbody > tr'))

In [8]:
total_page = 1
page = 1
result = {}
keys = ['증권종류','회사명','종목코드','업종','주요제품','상장일','결산일','대표자명','홈페이지','지역']
while page <= total_page:
    print(f"{page}/{total_page} 페이지 자료 수집중", end="\r")
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100,  orderMode=3, orderStat="D", searchType=13,fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
    soup = bs(r.content,'lxml')
    total_page = int(soup.select_one(".info.type-00 > em").text.replace(",","")) // 100 +1
    

    
    for tr in soup.select('tbody > tr'):
        # 증권종류
        types = tr.select_one(".vmiddle.legend")['alt']
        # 회사명
        name = tr.select_one("#companysum")["title"]
        # 종목코드
        code = tr.select_one("#companysum")["onclick"].split("'")[1]
        code = f'{code:0<6s}'
        #업종
        type_bi = tr.select_one(".textOverflow").text
        #주요제품
        main = tr.select_one("td:nth-child(2)").text
        # 상장일
        date = tr.select_one("td:nth-child(4)").text
        # 결산일
        sum_date= tr.select_one("td:nth-child(5)").text
        #대표자명
        ceo = tr.select_one("td:nth-child(6)").text
        #홈페이지
        link = tr.select_one("td:nth-child(7)>a")['href'] if tr.select_one("td:nth-child(7)>a") != None else '없음'
        # 지역
        do = tr.select_one("td:nth-child(8)").text


        values = (types,name,code,type_bi,main,date,sum_date,ceo,link,do)

        for key, value in zip(keys, values):
            result.setdefault(key, []).append(value)
    page += 1
df = pd.DataFrame(result)
df

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산일,대표자명,홈페이지,지역
0,유가증권,명인제약,317450,의약품 제조업,의약품 제조업,2025-10-01,12월,이행명,http://www.myunginph.co.kr,경기도
1,코스닥,KB제33호스팩,0072Z0,금융 지원 서비스업,금융 지원 서비스업,2025-09-30,12월,박성원,없음,서울특별시
2,코스닥,삼성스팩11호,0071M0,금융 지원 서비스업,금융 지원 서비스업,2025-09-30,12월,문소연,없음,서울특별시
3,코스닥,에스투더블유,488280,소프트웨어 개발 및 공급업,소프트웨어 개발 및 공급업,2025-09-19,12월,서상덕,http://s2w.inc/,경기도
4,코스닥,오아,342870,가전제품 및 정보통신장비 소매업,가전제품 및 정보통신장비 소매업,2025-09-09,12월,김상무,http://www.oa-world.com,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2758,유가증권,유한양행,000100,의약품 제조업,의약품 제조업,1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2759,유가증권,CJ대한통운,000120,도로 화물 운송업,도로 화물 운송업,1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2760,유가증권,경방,000050,종합 소매업,종합 소매업,1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2761,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,회사 본부 및 경영 컨설팅 서비스업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [10]:
df.to_csv("./data/상장기업정보.csv", index = False , encoding = 'utf-8-sig')